Chat 은 Chain과 Prompt 개념이 나오기 때문에,
Prompt, Chain 설명후에, ChatModel 설명하는게 좋겠음


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatOpenAI(model="gpt-4o-mini", api_key="{YOUR_OPENAI_KEY}")

messages = [
    SystemMessage(
        content="You are the travel agent. You can provide travel itinery to the user"
    ),
    HumanMessage(content="Where is the top 3 popular space for tourist in Seoul?"),
]

aiMessage = chat.invoke(messages)
print(aiMessage.content)
messages.append(aiMessage)
print("-" * 30)

messages.append(HumanMessage(content="Which transport can I use to visit the places?"))
aiMessage = chat.invoke(messages)
print(aiMessage.content)
print("-" * 30)

messages.append(
    HumanMessage(content="Where is the good restaurant for family near the placee?")
)
aiMessage = chat.invoke(messages)
print(aiMessage.content)

Seoul is a vibrant city with a mix of modern and traditional attractions. Here are the top three popular tourist spots in Seoul:

1. **Gyeongbokgung Palace**:
   - This is the largest of the Five Grand Palaces built during the Joseon Dynasty. Visitors can explore the beautiful architecture, serene gardens, and the National Palace Museum of Korea. Don't miss the changing of the guard ceremony that takes place at the main gate.

2. **N Seoul Tower (Namsan Tower)**:
   - Located on Namsan Mountain, this iconic tower offers panoramic views of the city. You can take a cable car or hike up to the tower. The observation deck is a popular spot for couples, and the area around the tower is filled with love locks, making it a romantic destination.

3. **Myeongdong Shopping Street**:
   - This bustling shopping district is famous for its street food, cosmetics shops, and fashion boutiques. It's a great place to experience the vibrant atmosphere of Seoul, try delicious Korean snacks, and shop for 

The chat model interface is based around messages rather than raw text. The types of messages currently supported in LangChain are AIMessage, HumanMessage, SystemMessage, FunctionMessage and ChatMessage – ChatMessage takes in an arbitrary role parameter. Most of the time, you’ll just be dealing with HumanMessage, AIMessage, and SystemMessage
<p>
chat model intro : https://blog.langchain.dev/chat-models/ <br>
Memory intro : https://nanonets.com/blog/langchain/#module-v-memory <br>
Memory Type : https://pub.towardsai.net/langchain-deepdive-memory-the-key-to-intelligent-conversations-783a23a780f9

# ChatMessage History

In [7]:
# ChatMessageHistory is used if you are managing memory outside of a chian directly
from langchain_community.chat_message_histories import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Where is the top 3 popular space for tourist in Seoul?")
aiMessage = chat.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)
print("-" * 20)

history.add_user_message("Which transport can I use to visit the places?")
aiMessage = chat.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)

Seoul offers a variety of attractions for tourists, but three of the most popular spaces are:

1. **Gyeongbokgung Palace**: This is the largest and most iconic of the Five Grand Palaces built during the Joseon Dynasty. Visitors can explore its beautiful architecture, gardens, and the National Palace Museum of Korea. The changing of the guard ceremony is a popular highlight.

2. **N Seoul Tower (Namsan Tower)**: Located on Namsan Mountain, this tower provides panoramic views of the city. Visitors can take a cable car or hike up to the tower, where they can enjoy dining, shopping, and the famous "Locks of Love" area, where couples attach padlocks as a symbol of their love.

3. **Myeongdong Shopping District**: This vibrant area is a shopping haven, known for its cosmetics shops, fashion boutiques, and street food stalls. It's a great place to experience the hustle and bustle of Seoul, try local snacks, and shop for souvenirs.

These locations offer a mix of cultural, historical, and mode

In [8]:
help(ChatMessageHistory)

Help on class InMemoryChatMessageHistory in module langchain_core.chat_history:

class InMemoryChatMessageHistory(BaseChatMessageHistory, pydantic.main.BaseModel)
 |  InMemoryChatMessageHistory(*, messages: list[langchain_core.messages.base.BaseMessage] = None) -> None
 |
 |  In memory implementation of chat message history.
 |
 |  Stores messages in a memory list.
 |
 |  Method resolution order:
 |      InMemoryChatMessageHistory
 |      BaseChatMessageHistory
 |      abc.ABC
 |      pydantic.main.BaseModel
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  async aadd_messages(self, messages: 'Sequence[BaseMessage]') -> 'None'
 |      Async add messages to the store.
 |
 |      Args:
 |          messages: The messages to add.
 |
 |  async aclear(self) -> 'None'
 |      Async clear all messages from the store.
 |
 |  add_message(self, message: 'BaseMessage') -> 'None'
 |      Add a self-created message to the store.
 |
 |      Args:
 |          message: The message to add.
 |


# ConversationBufferMemory

In [9]:
print(history.messages)

[HumanMessage(content='Where is the top 3 popular space for tourist in Seoul?', additional_kwargs={}, response_metadata={}), AIMessage(content='Seoul offers a variety of attractions for tourists, but three of the most popular spaces are:\n\n1. **Gyeongbokgung Palace**: This is the largest and most iconic of the Five Grand Palaces built during the Joseon Dynasty. Visitors can explore its beautiful architecture, gardens, and the National Palace Museum of Korea. The changing of the guard ceremony is a popular highlight.\n\n2. **N Seoul Tower (Namsan Tower)**: Located on Namsan Mountain, this tower provides panoramic views of the city. Visitors can take a cable car or hike up to the tower, where they can enjoy dining, shopping, and the famous "Locks of Love" area, where couples attach padlocks as a symbol of their love.\n\n3. **Myeongdong Shopping District**: This vibrant area is a shopping haven, known for its cosmetics shops, fashion boutiques, and street food stalls. It\'s a great place

In [14]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", api_key="{YOUR_OPENAI_KEY}")

store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain = RunnableWithMessageHistory(chat, get_session_history)
resp = chain.invoke(
    "Where is the top 3 popular space for tourist in Seoul?",
    config={"configurable": {"session_id": "1"}},
)
print("RESPONSE", resp)
resp2 = chain.invoke(
    "Which transport can I use to visit the places?",
    config={"configurable": {"session_id": "1"}},
)
print("RESPONSE", resp2)

RESPONSE content='Seoul offers a variety of attractions for tourists, but three of the most popular spaces are:\n\n1. **Gyeongbokgung Palace**: This is the largest and most iconic of the Five Grand Palaces built during the Joseon Dynasty. Visitors can explore its beautiful architecture, gardens, and the National Palace Museum of Korea. The changing of the guard ceremony is a popular highlight.\n\n2. **N Seoul Tower (Namsan Tower)**: Located on Namsan Mountain, this tower provides panoramic views of the city. Visitors can take a cable car or hike up to the tower, where they can enjoy dining, shopping, and the famous "Locks of Love" fence.\n\n3. **Myeongdong Shopping District**: This vibrant area is known for its shopping, street food, and beauty products. It\'s a bustling hub where tourists can experience Korean culture, try local delicacies, and shop for cosmetics and fashion.\n\nThese locations offer a mix of history, culture, and modern attractions, making them must-visit spots for a